In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [3]:
from CoreConfig.CoreConfig import CoreConfig
from DBUtility.DBUtility import DBSQLiteConnection, CreateEventsTableIfNotExist, CreateNPSTableIfNotExist
from DBUtility.DBUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson, FetchEventsRecordAll, FetchNPSRecordAll
from DBUtility.DBUtility import InsertEventRecord, InsertNPSRecord
from pandas import DataFrame
import datetime


import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
from StylingUtility.StylingUtility import CustomStyling

# CoreConfig.initialize_core_config()
# DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
# CreateEventsTableIfNotExist().execute()


### NPS Form

In [138]:
# get configuration details
CoreConfig.initialize_core_config()

# establish database connection
DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())

# create table if not exist
CreateNPSTableIfNotExist().execute()

Initialize nab NPS table failed. Please check your connection
table nab_mobile_nps already exists


In [149]:
title_nps = widgets.HTML("<center><H3>Mobile NPS Survey</H3><center>")

formtitle_nps = widgets.HTML("<center><H3>Digital Channel - Internal Survey</H3><center>")

df_out_nps = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})

with df_out_nps:
    df = DataFrame(FetchNPSRecordAll().execute(), columns=['Date', 'Channel', 'Score', 'Thoughts', 'Comments'])
    display(df)

date = widgets.DatePicker(
        value = datetime.datetime.now().date(),
        description='Pick a Date',
        layout=CustomStyling.box_layout,
        style = CustomStyling.style,
        disable=False)
    
channel = widgets.Dropdown(options= {'NAB Connect': 'NAB Connect',
                                     'NAB Connect Mobile': 'NAB Connect Mobile',
                                    'NAB Internet Banking':'NAB Internet Banking',
                                    'NAB Mobile App': 'NAB Mobile App',
                                    'NAB.com.au': 'NAB.com.au'},
                           value = 'NAB Mobile App',
                           description='Select channel:',
                           layout=CustomStyling.box_layout,
                           style=CustomStyling.style)

q1 = widgets.IntSlider(
        value=7,
        min=0,
        max=10,
        step=1,
        description='How likely are you to recommend working in the Mobile Service to others?:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        layout=CustomStyling.box_layout,
        style=CustomStyling.style,
        readout_format='d')
    
q2 = widgets.Text(
      value='',
      placeholder='Type Here',
      description='Share your thoughts on how can we improve the Mobile Service?',
      layout=CustomStyling.box_layout,
       style=CustomStyling.style,
      disable=False)

q3 = widgets.Text(
      value='',
      placeholder='Type Here',
      description='Any additional comments?',
      layout=CustomStyling.box_layout,
       style=CustomStyling.style,
      disable=False)

button_nps = widgets.Button(
           description='Submit the form',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_submit_form_button_clicked_nps(b):
    outputs = { 'date' : [date.value],
                'channel' : [channel.value],
                'q1' : [q1.value],
                'q2' : [q2.value],
                'q3' : [q3.value]
               }
    InsertNPSRecord(date.value, channel.value, q1.value, q2.value, q3.value).execute()
    df = DataFrame(FetchNPSRecordAll().execute(), columns=['Date', 'Channel', 'Score', 'Thoughts', 'Comments'])
    df_out_nps.clear_output()
    with df_out_nps:
        display(df)
        
button_nps.on_click(on_submit_form_button_clicked_nps)

In [150]:
# user interface organised using VBox (vertical)
ui_nps = widgets.VBox(children=[title_nps,
                            df_out_nps,
                            formtitle_nps,
                            date,
                            channel,
                            q1,
                            q2,
                            q3,
                            button_nps], layout=CustomStyling.items_layout)

# live display of values, for debugging purposes
def form_nps(date, channel, q1, q2, q3):
    display(date, channel, q1, q2, q3)
    return date, channel, q1, q2, q3

out_nps = interactive_output(form_nps, {'date': date,
                                'channel': channel,
                                'q1' : q1,
                                'q2':q2,
                                'q3':q3
                               })

display(ui_nps, out_nps)

A Jupyter Widget

A Jupyter Widget

In [ ]:
out_nps.outputs

### Prototype - Update Record

In [209]:
from DBUtility.DBUtility import CreatePOCTableIfNotExist, InsertPOCRecord, FetchPOCRecordAll, DropTable, UpdatePOCRecord, GetPOCStatus

In [211]:
# get configuration details
CoreConfig.initialize_core_config()

# establish database connection
DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())

# create table if not exist
CreatePOCTableIfNotExist().execute()

Initialize nab POC table failed. Please check your connection
table nab_poc_status already exists


In [183]:
DropTable('nab_poc_status').execute()

In [185]:
InsertPOCRecord('POC_1', 'Old_Status').execute()
InsertPOCRecord('POC_2', 'Old_Status').execute()
InsertPOCRecord('POC_3', 'Old_Status').execute()
InsertPOCRecord('POC_4', 'Old_Status').execute()
InsertPOCRecord('POC_5', 'Old_Status').execute()

In [133]:
UpdatePOCRecord('POC_1', 'New_Status').execute()

In [219]:
df_out_poc = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})

with df_out_poc:
    df = DataFrame(FetchPOCRecordAll().execute(), columns=['POC_Name', 'POC_Status'])
    display(df)


POC_dict = {}
POC_list = FetchPOCRecordAll().execute()
for name in [row[0] for row in POC_list]:
    POC_dict[name] = name
    
POC_Name = widgets.Dropdown(options= POC_dict,
                           value = 'POC_1',
                           description='Change status for:',
                           layout=CustomStyling.box_layout,
                           style=CustomStyling.style)

default_status = 'New_Status'
Status_Option = widgets.Dropdown(options= {'Old_Status': 'Old_Status',
                                     'New_Status': 'New_Status'},
                           value = default_status,
                           description='Update status to:',
                           layout=CustomStyling.box_layout,
                           style=CustomStyling.style)





button_to_update_form = widgets.Button(
                       description='Update Form',
                       disable=False,
                       button_style ='', # success
                       tooltip='Click me',
                       icon='check'
                       )

def on_update_form_button_clicked_poc(b):
    default_status = GetPOCStatus(POC_Name.value).execute()
    Status_Option.value = default_status


button_to_update_form.on_click(on_update_form_button_clicked_poc)







button_poc = widgets.Button(
           description='Update Status',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_submit_form_button_clicked_poc(b):
#     outputs = { 'POC_Name' : [POC_Name.value],
#                 'Status_Option' : [Status_Option.value]
#                }
    UpdatePOCRecord(POC_Name.value, Status_Option.value).execute()
    df = DataFrame(FetchPOCRecordAll().execute(), columns=['POC_Name', 'POC_Status'])
    df_out_poc.clear_output()
    with df_out_poc:
        display(df)
        
button_poc.on_click(on_submit_form_button_clicked_poc)





# user interface organised using VBox (vertical)
ui_poc = widgets.VBox(children=[df_out_poc,
                            POC_Name,
                            button_to_update_form,
                            Status_Option,
                            button_poc], layout=CustomStyling.items_layout)

# live display of values, for debugging purposes
def form_poc(POC_Name, Status_Option):
    #display(POC_Name, Status_Option)
    return POC_Name, Status_Option

out_poc = interactive_output(form_poc, {'POC_Name': POC_Name,
                                'Status_Option': Status_Option
                               })

display(ui_poc, out_poc)

A Jupyter Widget

A Jupyter Widget

### Experiment

In [ ]:
a = widgets.IntSlider(description='a', disabled=True)
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [ ]:
check_exper = widgets.Checkbox(
    value=False,
    description='Add new person',
    disable = False)
person_text_exper = widgets.Text(
      value='',
      placeholder='Type your name',
      description='New person to be added:',
      layout=box_layout,
       style=style,
      disabled=True)
display(check_exper, person_text_exper)

def on_value_change(change):
    if change['new'] == True:
        person_text_exper.disabled=False
    else:
        person_text_exper.disabled=True

check_exper.observe(on_value_change, names='value')

In [ ]:
expr_country = widgets.Dropdown(options= {'Australia': 'Australia',
                                     'New Zealand': 'New Zealand',
                                    'None':'None'},
                                   value = 'Australia',
                               description='Select country:',
                               layout=box_layout,
                               style=style)
display(expr_country)

In [ ]:
w = widgets.IntSlider()
display(w)
display(w)